In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec  4 10:57:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!apt-get install libsndfile1

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.c

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0dfuod00
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-0dfuod00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 6.4 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5858215 sha256=020dae86dc7c10ab999b22cb956604f68e2da78c17815b6890a653268f2cf644
  Stored in directory: /tmp/pip-ephem-wheel-cache-c1mtgbvd/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
processor = AutoProcessor.from_pretrained("openai/whisper-small")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
!wget https://storage.googleapis.com/test_public_bucket/labelled/Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
!unzip Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip

--2022-12-04 10:59:56--  https://storage.googleapis.com/test_public_bucket/labelled/Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90658931 (86M) [application/zip]
Saving to: ‘Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip’

Telugu_Chai_Bisket_ 100%[===================>]  86.46M  70.0MB/s    in 1.2s    

2022-12-04 10:59:57 (70.0 MB/s) - ‘Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip’ saved [90658931/90658931]

Archive:  Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
   creating: Chai_Bisket_Stories_16-08-2021_14-17/
  inflating: Chai_Bisket_Stories_16-08-2021_14-17/3_1_481file-idc4p8tJhhC0o.wav  
  inflating: Chai_Bisket_Stories_16-08-2021_14-17/10_16_596file-idIErLEb3RPs4.wav  
  inflating: Chai_Bisket_Stories_16-08-20

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict, Dataset, Audio

ds_1 = load_dataset("/content/Chai_Bisket_Stories_16-08-2021_14-17")

ds_1 = ds_1['train']
print(ds_1)

ds_2 = load_dataset("json",data_files="/content/Chai_Bisket_Stories_16-08-2021_14-17/data.json")
ds_2 = ds_2['train']
print(ds_2)

final_dataset = datasets.concatenate_datasets([ds_1, ds_2], axis=1)
print(final_dataset)

train_test_dataset = final_dataset.train_test_split(test_size=0.30)
print(train_test_dataset)


train_test_dataset = train_test_dataset.remove_columns(['audioFilename','collectionSource', 'snr', 'duration', 'gender',])
print(train_test_dataset)

ModuleNotFoundError: ignored

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

In [ ]:
train_test_dataset['test']

Dataset({
    features: ['audio', 'text'],
    num_rows: 176
})

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="bnriiitb/whisper-small-te")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
!pip install -U tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_test_dataset['test']['audio'][0]['array']

array([-0.00085449, -0.01168823,  0.0083313 , ...,  0.00042725,
        0.00094604,  0.00097656], dtype=float32)

In [ ]:
import time
from tqdm import tqdm

finetuned_texts = []

for item in tqdm(train_test_dataset['test']['audio']):
  finetuned_texts.append(pipe(item['array'])['text'])

finetuned_texts

  0%|          | 0/176 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 176/176 [58:08<00:00, 19.82s/it]


['అంటింది పెప్పింది కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు',
 ' So what happens is this uncle is there now this uncle kills the family so what do you want this',
 " I don't know what to say.",
 'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
 ' Thank you',
 ' What are you doing?',
 " Tell me what's the most commonly used phrase in Bangalore? Exactly, use the same strategy here also.",
 " That's the matter",
 ' I am very happy to be a part of this. I wish you all the best.',
 ' Agent Bobby speaking.',
 ' Engineering is the most important software engineer in the world. Life is a very important thing.',
 'అంటి పెప్పి పార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి 

In [ ]:
import whisper
whisper_small_model = whisper.load_model("small")

whisper_langs = []
whisper_texts = []

for item in tqdm(train_test_dataset['test']['audio']):
  pred = whisper_small_model.transcribe(item['array'])
  whisper_texts.append(pred['text'])
  whisper_langs.append(pred['language'])
  

whisper_texts,whisper_langs

100%|██████████| 176/176 [29:35<00:00, 10.09s/it]


([' अद्दिने इद्दार अफिसले केल पोटर। वेगासी निन नेन। अम्मा, इदे नी फाणेड़सिशना? अवन। अगर देंगे जाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजा�',
  ' So what happens is this uncle is there no this uncle kills the family so Bar is to do of the other family so what do you want is',
  ' ಸಿಲಿರಿಯಾರಂದ ಮಾಡಿಕಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ�',
  ' వారినిక్లినినిక్లినినినిందినిందిని. ఇసిచియానిమిలానికందికందినినికిలినినినిని.',
  ' मालविकाल',
  ' Как десеный!',
  " Tell me what's the most commonly used phrase in Bangalore? It's the same phrase in Bangalore. Exactly! Use the same strategy here also. I don't know the line between us.",
  ' अदि मैंटर',
  ' కాట్న్గారినింద్నినిందారం కాట్న్న్న్నిందారింద్నిందారిందారింద్నిందారిందారిందారిందారిందారిందారిందారిందారిందారిందార�',
  ' Agent Bobby speaking.',
  ' ఇరిల్ ఎనిల్లు పిను ఇరిల్లు పినులు గార్లిలు ను మిల్లు నిల్లు మార్ల

In [ ]:
label_str = train_test_dataset['test']['text']
label_str[:2]

['ఇంకా అక్కడి నుండి వాళ్ళు వెంటనే చెప్పుకుంటారు',
 'మీ గ్రీక్ సోల్జర్ లా ఫీల్ అయ్యి బయలుదేరి']

In [ ]:
finetuned_wer = 100 * metric.compute(predictions=finetuned_texts, references=label_str)
whisper_wer = 100 * metric.compute(predictions=whisper_texts, references=label_str)
whisper_wer,finetuned_wer

(173.7108190091001, 442.9726996966633)

In [ ]:
# curl -X GET \
#      "https://datasets-server.huggingface.co/first-rows?dataset=google%2Ffleurs&config=te_in&split=train"

In [ ]:
# curl -X GET \
    #  "https://datasets-server.huggingface.co/first-rows?dataset=librispeech_asr&config=clean&split=train.100"
# librispeech_eval_1 = load_dataset("google/fleurs", "te_in", split="train")     
# librispeech_eval_1

In [ ]:
# librispeech_eval_1 = librispeech_eval_1.rename_column('transcription','text')
# librispeech_eval_1

In [ ]:
# from datasets import load_dataset
# from transformers import WhisperForConditionalGeneration, WhisperProcessor
# import soundfile as sf
# import torch
# from evaluate import load


# # librispeech_eval = load_dataset("librispeech_asr", "clean", split="test")

# librispeech_eval = train_test_dataset['test']

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# def map_to_pred(batch):
#     input_features = processor(batch["audio"][0]["array"], return_tensors="pt").input_features

#     with torch.no_grad():
#         logits = model(input_features.to("cuda")).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = processor.batch_decode(predicted_ids, normalize = True)
#     batch['text'] = processor.tokenizer._normalize(batch['text'])
#     batch["transcription"] = transcription
#     return batch

In [ ]:
# train_test_dataset['test']['audio'][0]['path']

In [ ]:
# ['అంటింది పెప్పింది కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు',
#  ' So what happens is this uncle is there now this uncle kills the family so what do you want this',
#  " I don't know what to say.",
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  ' Thank you',
#  ' What are you doing?',
#  " Tell me what's the most commonly used phrase in Bangalore? Exactly, use the same strategy here also.",
#  " That's the matter",
#  ' I am very happy to be a part of this. I wish you all the best.',
#  ' Agent Bobby speaking.',
#  ' Engineering is the most important software engineer in the world. Life is a very important thing.',
#  'అంటి పెప్పి పార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార్పి కార',
#  ' Is it a homework? Yes Sir! Good!',
#  ' and they are not only Pakistan, but also all over the world',
#  'కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు క',
#  'అంటింది పార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కా',
#  ' Experimentally',
#  'అంటింది పార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి ',
#  ' But the one who is not there is not the one who is going to go',
#  ' and the people of Khyangar',
#  ' World Cup final Guruji, Ghusbham Se Chai Rein Chlo Yala Gaina Chaapachu But I wanted you to take something from this episode',
#  'అంటింది కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాదు కాద',
#  ' I am not a person',
#  ' and the people will be happy.',
#  ' 499. Yes.',
#  'అంటింది పార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కా',
#  'అంటింది పార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కా',
#  'అంటింది పరింది కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క',
#  ' "I am not a person who can be seen in the mirror, but I am a person who can be seen in the mirror."',
#  ' This is the first time I have seen a person like him',
#  'అంటింది పార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కా',
#  " If you face this problem, then what will happen? If you don't want to face it, then you can't do it",
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  ' What?',
#  'కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు క',
#  'అంటింది పార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కా',
#  'కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగుండి కాగు�',
#  'అంటి పార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార',
#  " So, let's start.",
#  'అంటి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార',
#  ' I am not sure',
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  'అంటి పెప్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార',
#  'అంటి పెప్పి కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్',
#  'అంటి కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార',
#  ' Okay.',
#  " It's a very long story",
#  ' I am not going to go',
#  ' My life is still perfect.',
#  ' I am not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is a person who is not a person who is not a person who is a person who is not a person who is not a person who is a person who is not a person who is not a person who is a person who is not a person who is a person who is not a person who is a person',
#  ' I am not sure.',
#  ' I',
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  ' If you are not a Guru, you can go to the temple and pray.',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  'కాగుంది కాగుంది కాగుంది కాగుంది కాగుంది కాగుంది కాగుంది కాగుంది కాగుంది కాగుంది',
#  'అంటింది పెప్పండి కార్సింది కార్సింది',
#  ' This is the first time I have seen a game like this.',
#  " I'm not going to let you go.",
#  ' What is the difference between the two? What is the difference between the two?',
#  ' I will not let you go',
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  " Let's start",
#  ' and the reason for the death of the family is that the family was separated from the family',
#  'అంటి కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార',
#  ' Great things take time.',
#  ' This is the place where the people of Vandhapur come',
#  ' I am not sure if it is because of the rain or the rain.',
#  ' It is a very important thing to remember',
#  'అంటింది కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కాన',
#  ' This is the story of the new generation.',
#  ' I was a little scared.',
#  'అంటింది పారుగాంది కారుగాంది కారుగాంది కారుగాంది కారుగాంది',
#  'కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా కాగా క�',
#  ' See you',
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  ' I think it was around 8am when I woke up. I was very tired and I was very tired.',
#  ' The most important thing is to keep the place clean.',
#  ' I will be back',
#  ' I am not sure if it is true',
#  'అంటి పెప్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార్పం పార',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  'అంటింది పెప్పింది కార్పింది కార్పింది',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  ' I am not a person who can understand the meaning of words',
#  ' [Speaking in Hindi]',
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  ' How many people are there? 30-41 people, they are very good. They have been here for 6 hours. They are very good. They are very good.',
#  ' Yeah.',
#  ' After few days, I went to college',
#  ' Opportunity in adversity',
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  'అంటి పెప్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి',
#  " Hey, why are you crying? It's because of the comedy. It's because of the emotions. I don't like emotions.",
#  ' Happy birthday',
#  'కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు కాగు క�',
#  " I don't know why I'm here.",
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  " Okay, let's take attendance now.",
#  ' I am not a person who is not interested in anything. I am a person who is not interested in anything.',
#  ' Adhi matter',
#  'అంటి పెప్పి కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్ కార్స్',
#  'కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు క',
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  'పాప్పుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు',
#  ' See you soon',
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  'అంటింది పార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కార్లారు కా',
#  ' I am not saying that I am not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a person who is not a',
#  'అంటింది పార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి కార్లి ',
#  'కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు క',
#  'అంటింది పార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కార్లాలు కా',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  ' The same goes for the religious aspect',
#  " I don't know what you are talking about. Sir, I don't know what you are talking about. I don't know what you are talking about.",
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  ' [coughing]',
#  'అంటింది పర్పండి కార్పండి కార్పండి కార్పండి కార్పండి కార్పండి కార్పండి కార్పండి',
#  ' I am not sure if I will be able to do it.',
#  'అంటింది పెప్పింది కార్సింది కార్సింది కార్సింది కార్సింది',
#  'ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప',
#  ' The reason for this is that the people of the world are not only the people of the world, but also the people of the world.',
#  ' See you on Sunday',
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  'పాప్పుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు పాపుడు',
#  'అంటింది పరించి కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్లాలో కార్',
#  'అంటింది కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో కార్లో ',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  " I know, it's romantic to see you as a girl. But I think it's romantic.",
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  ' This is the story of the two women who were born in the same village.',
#  'అంటి పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార్లో పార',
#  ' I am not sure if it is true or not.',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  " I don't know how I feel. I feel like I'm in heaven.",
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  " So it's something bigger than any person in your life",
#  ' This is the place where the birds are flying',
#  ' and I am not sure',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  'అంటింది పార్లాంది కార్లాంది కార్లాంది కార్లాంది కార్లాంది కార్లాంది',
#  " I don't know what I'm talking about.",
#  'ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప్పుడు ప',
#  'అంటింది పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పార్లాలో పా',
#  " It's a very different feeling",
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  " So, I think that's the reason why I chose this place.",
#  'కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు క',
#  ' Did I say anything wrong? No, I asked him to become more responsible',
#  " Who is this? All of this is being recorded. Okay then, let's start the class. You people should take down the notes.",
#  " I don't know what to say",
#  ' See you again',
#  " It's a very interesting movie.",
#  ' I am not sure if it is the MG TV or the Gola TV. I will have to choose one.',
#  " So, I think that's it.",
#  'కాగుంటి కాగుంటి కాగుంటి కాగుంటి కాగుంటి కాగుంటి కాగుంటి కాగుంటి',
#  'అంటింది పర్పండి కార్పండి కార్పండి కార్పండి కార్పండి',
#  ' I will see you in the next video',
#  ' I want to see the movie promotion. What is the promotion? Okay, okay, sorry. So, what happened? What did you say? I said that I will not go anywhere.',
#  ' Where are you going? Where are you going? Where are you going? Where are you going?',
#  " It's not like I'm going to leave you alone. I'm not going to leave you alone. I'm not going to leave you alone. I'm not going to leave you alone. I'm not going to leave you alone. I'm not going to leave you alone.",
#  " So, this is the coffee shop. This is the kitchen. This is the kitchen. This is the kitchen. This is the kitchen. So funny. It's a separate kitchen.",
#  'అంటి పెప్పం పారుగాంటి పారుగాంటి పారుగాంటి పారుగాంటి పారుగాంటి పారుగాంటి పారుగాంటి',
#  ' This is the story',
#  'అంటి పెప్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి పార్పి',
#  ' They were not only military soldiers, but also...',
#  'అంటింది కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కాన',
#  'కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ కానీ క�',
#  ' Thank you',
#  'కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్లు కార్రు కార్రు క',
#  ' This is the story of Agra Tambula',
#  ' and her smile',
#  'అంటింది పెప్పండి కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు కార్స్తారు',
#  " I don't know what's going on."]

In [ ]:
# [' अद्दिने इद्दार अफिसले केल पोटर। वेगासी निन नेन। अम्मा, इदे नी फाणेड़सिशना? अवन। अगर देंगे जाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजाएजा�',
#   ' So what happens is this uncle is there no this uncle kills the family so Bar is to do of the other family so what do you want is',
#   ' ಸಿಲಿರಿಯಾರಂದ ಮಾಡಿಕಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ�',
#   ' వారినిక్లినినిక్లినినినిందినిందిని. ఇసిచియానిమిలానికందికందినినికిలినినినిని.',
#   ' मालविकाल',
#   ' Как десеный!',
#   " Tell me what's the most commonly used phrase in Bangalore? It's the same phrase in Bangalore. Exactly! Use the same strategy here also. I don't know the line between us.",
#   ' अदि मैंटर',
#   ' కాట్న్గారినింద్నినిందారం కాట్న్న్న్నిందారింద్నిందారిందారింద్నిందారిందారిందారిందారిందారిందారిందారిందారిందారిందార�',
#   ' Agent Bobby speaking.',
#   ' ఇరిల్ ఎనిల్లు పిను ఇరిల్లు పినులు గార్లిలు ను మిల్లు నిల్లు మార్లు మివర్లు మార్లు మోందిక్రిక్రి మార్లు మార్లు మార్లు మా',
#   ' ఇటార్న్ ఇటిటినిర్న్ిమున్న్లున్లులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులుల�',
#   ' अंदर होमर्क चेसरा? येस पार शालक येसeniz की ताकरते क्यट सी纬ाग झाड़्टیटा क्यट तो अग scène कँर अई कील वाभीOOM',
#   ' आस्टेल्या आईना पाकिस्टान आईना प्दददेडा लेदन् कोवालि',
#   ' பாண்ணி இருக்கதே?',
#   '있는 なの?',
#   ' அனைத்துக்கொள்ளுங்கள்.',
#   ' వారినినినిని నిందినినినిని నినినినినినినినిని. పారినినినిని నినినినినిని. నినినినినినినినినినినినినినినినినినినినినిని.',
#   ' ట్ట్రు దువ్ట్ల్ త్ఇ్ప్లోను నివ్లినార్ల్ల్ల్ల్ల్ నిస్ల్ల్ర్ల్లిల్ల్ల్ల్.',
#   ' నిన్లు నిలుని నిస్ల్లు నిస్లు ప్లు నిన్లులు నిస్లులు స్లులు నిస్లు నిస్లు నిస్లు స్లులు నిస్లు స్లులు స్లులు స్లులు స్ల',
#   ' वोल्काप पार्णल गुर्ची गुस्पर्म से चार्य रेंज लोग यलागाने चब पच्वूँ जब आप आप आप आप वाड़ाट कर वहाँ वाँगाने वाए वाँगागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागागा',
#   ' తారికాకికార్టారానిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింది.',
#   ' seems like they have not failed yet',
#   ' ఇరిని కివర్చి పరినినిస్ది దివర్చి నినింది నిని ప\u0c70ినినిని నినినినిని నినింది నినిని నినిని నినినిని నినినిని నినినిని న',
#   ' 499 yes 599',
#   ' కానినినినిని ఉథినినినిలానినిని. మానినినినినినినినినినినినినినినినినినినినినినినినినినినిని.',
#   ' च्वरि किया वरु गेलोवा को पुर्डम तो',
#   ' विटनिद तो पातू',
#   ' ताना यूवराजू खानी येदो थिल्यानी अनुमाना अंदरू तना गुरिंची तकुगा मात लागुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगुटारेमो अनिएगु�',
#   ' వారిల్నినినినినినినినినినినిన౿నినినినినినినినినిని.',
#   ' నినినినిని మారినిని నినిని నిన౿నినినిసిని నినినిని నినినినినినినినిని నినినినినినినినినినిని నినినినినినినిని మినిన',
#   ' వర్ర్త్ర్ర్న్నిందిందిందిందిందిందింది పెందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింది�',
#   ' වස මිනියින්ම්න්ල් ලාපුරයින්න්ල්ලාවන් පුරයින්න්ල්ලාලාවන්වන්ලාන්ලායින්ල්ලාන්ලාවන්ලාන්ලක්ලාන්ලාන්ලාන්න්ලාන්න්නාන්වන්න',
#   ' ��두... కారంనాందారార్లిన కటారంతాలిందాందాల౿ందార్లిందారిందాలిందాందిందాందాలిందాలిందాలిందాలిందాలిందాలిందాలి కటిలిందిందా కటాల�',
#   ' సిన్టినినినేనినినినిందిందినిందినునునినినిందినినినులినినినినినినినినినినినునినినినినినినినినినిన౿నినినినినినినిన',
#   ' మార్నివార్లు నినిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిం�',
#   ' यो रोज मा पोट्काश तांगितो.',
#   ' వార్లా గావావార్లావార్లినినులినినులినిరినినినిని. అలికికార్లినినినిని. వినినినినినినినినినినినినినినిని.',
#   '',
#   ' వారికింరినినింరిందిని నినింది నిరింది నిందిందింది నిందిందిందింది నిందింది నిందింది నింది నిందింది నింది నింది నింది న',
#   ' ஆனால் அதே பெரு',
#   ' 인개로는 비싼데',
#   ' अदि लाक गंडा आप साप योंचे दूर लोपल किलते',
#   ' अप तो प्रद च्वूड यूग लोपला गुवे डान्स लें अप उग्साम आप एक गर तो बगर देखा पाट गर तो वो बचाए बचाए तो बचाए तो वो वो बचाए पाट गर गर थागा पाट गर थागा बचाए पाट गर थागा पाट वो प्रद गर पाट गर गर गर थागा पाट गर गर गर गर पाट गर गर गर गर थागा पा�',
#   ' వార్న్లిలార్లినిక్ట్నిందానినినిందానిందానిందాందానిందానిందానిందానిందానిందానిందానిందానిందిందానిందానిందానిందానిందాన',
#   ' విలు నింది విలు నిక్త్లు పిన్లు పిలుక్లులు మ్లిక్లిలు స్లులులు ప్లులు మ్లులు మ్లులు మ్లులు మ్లులు మ్లులు మ్లులు మ్లిలు�',
#   ' � AG Flens is very cleansing.',
#   ' நான்தைக் கிருந்து சேர்',
#   ' నినిలి ప్లుటిందినిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింది�',
#   ' వారితికికితూట్నింటినిన్టింటింటింటింటిల్టింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటింటిం',
#   '',
#   'ächux స్ర్యోన్యాల్యాస్ల్యోల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్.',
#   ' కుర్నిల్ల్కర్తుర్నినినిందిందిందాట్లులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులుల',
#   ' අදු අපි කර්න් දැන් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න් කර්න්න් කර්න්න් කර්න්න් කර්න්න් කර්න්න් කර්න්න�',
#   ' बीश्मन की गुरुवेंग परसुराम। इवद्धक वेलि सहाई मडग मनर। परसुराम। गर गुरुवेंग वेलि सहाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई मड़ाई म�',
#   ' వార్న్టా క్నిల్కార్ల్రిన్న్స్య్ల్సిందినిల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల�',
#   ' Soripenne',
#   ' ಸಿಲಿಕಾರಿಕಿದಿವಿದಿದಿಡಾರಿನಿಲಿಲಿದ ಮಾದಿಗಾರಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ ನಿವಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ ಲಾಲಿ�',
#   ' వార్ట్ని నారిన్ని నినిక్నిక్నినిన్ని.',
#   ' தானுக்கு உடன நான் சல்லுவில்',
#   ' న్న్స్లు నిస్న్య్నినిన్నిన్నినినినినినినిని. నేలినినినినినెనినినినానేనినినినిని.',
#   ' நப்பு ராகப்பை நான்…',
#   ' ḍients for Lettia',
#   ' వార్న్చిన్చిన్చినిందిందిందింది. నిన్చిందిందింది. మారూతూదిందింది. పరినునానింది. నినిందిందిందింది. పరినానింది. మార్నునింది. మార్నునింది. మార్నునునును. మార్నును.',
#   '',
#   ' కాత్ట్తా కాత్ట్నిన్నిక్నిక్చ్త్పనిల్నిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిం�',
#   ' great things take time',
#   ' కాకానికానినిందిందింది.',
#   ' మ్ర్రిక్న్నాల్నినియానిల్నివర్న్నినిల్నినిల్నాన్టర్నినిల్నిల్న్నిందిల్నిల్నిల్న్నిల్నిల్న్నిల్నిల్నిలిల్నిల్నిల్',
#   ' నినికారారిని నినినినినిలినూటినూటినిని పిమారారినినినినినినినినినినినిన౿నినినిని.',
#   ' వాసాను నుతాసాను ఆర్ను ఆర్ను పిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింద',
#   ' గారిలి నిలి సిసిలాలిమూవారినినికినినినిని నిలిలిల౿నిల౿నిలినినిని.',
#   ' మారినారినింది మారినింది మారినినింది మారినినింది మారినింది మారినినినిమారిని పనినింది పని నినినినిని మారినినినింది మార',
#   ' Sampadish Nangadai',
#   ' Emo?',
#   ' अदिस कोचेरू',
#   ' ఎముపాలి డరెలోల్యవా.',
#   ' zdrow',
#   ' నివార్టార్నిందింది. మార్టిందింది మార్టింది. సిమార్టార్టింది. సిమార్టింది. సినిసింది. సిసిందింది. సిస్లందింది. సిసింది. �',
#   ' కారిన్న్నినారారినింద్నినినిందిన్నినింద్నినిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింద',
#   ' గారార్న్త్త్న్లిన్త్న్త్సిందిందిన్న్త్న్న్న్త్న్న్న్న్న్న్త్న్న్ని.',
#   ' मरी अलानो अर्धंकालेद।',
#   ' గార్ట్నికుస్ట్నినినికుత్రండికురార్నికునినానికునికికికునికికికునినికికికుని.',
#   ' నినినినినినినినూనినినినినినినిని.',
#   ' మారింటి నినినిక్చినింటినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినిని�',
#   ' పిమాని నికి సింది కారిని కికి క౿కి కికి పికికి మిక౿ కికి కికి కికి కికి కికి కికి కికి కికి కికి కికి కికి కికి కికి కికి �',
#   ' నోత్త్రి స్త్న్నిన్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్ల్లుల్ల్ల్ల్ల్ల్ల్ల్ల్లాన్ల్ల్�',
#   ' తారానానినిందింది ఇరిలినాతిలూతిందిలిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిం�',
#   ' == Ezhbal turnover Bloody hell!',
#   ' నినియార్నికార్నిలెరార్నినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినినిని.',
#   ' నార్లులు మార్లులులులు మార్లులులులులులాలులులులులులులులులులులులులులులిలులులులులులులులులులులులులులులులులులులులులులు',
#   ' మార్వా మారారిన్ మారినిన్యారానానినారినందార్ల్ల్ల్ల్లులుల్లులులులులులులులులులిలులులులులులులులులులులులులులులులులులు',
#   ' देवडा नु वुन्नाव देवडा अप्रुचिन्टी एं अद्वर्स्टेयन तु अग्वाव प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आप प्रुच्टी आ',
#   ' వాన్లియారిందార్ందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింది',
#   ' వారినినిసి పారినిందినిని పారిని. పారినినినిని. పారినినినిని నినినిందిని. పారినినిని. పారినినినినిని. పారినినినిని. పార�',
#   ' వార్నినినిందినిందినిందినిందినిందిందిందింద౿ంద౿ందిందిందిందిందిందిందిందిందిందిందిందిందిందింద౿ందిందిందింది.',
#   ' Hey, bye, buddy! Happy birthday! Thank you! Hey!',
#   ' 지',
#   ' గారాలులులులులులులులులులినినిస్వినినినినినినిని.',
#   ' a big curtain to cover in a place entrance.',
#   " Okay, let's take attendance now. Maam, na role number 5 maam, ko chum attendance iskond maam. First please, na data hai pohthund maam. Please maam. Roll attendance isn't maam. Wait, wait, wait. వార్లినినినిని మార్లినినినినినినినిక౿నినినినిల౿నిని నినినినినినినినినినినినినినినినినినినిన౿నినినినినినినినినిని",
#   ' मरेवरा या मिगलनवार की तकूकादू प्राएत में स्ते चवरी की सादिंचा वो चुप आप प्राएत में स्ते चवरी की सादिंचा वो चुप आप प्राएत में स्ते चवरी की सादिंचा वो चुप आप प्राएत में स्ते चवरी की सादिंचा वो चुप आप प्राएत में स्ते चवरी की सादिंचा वो चुप आ',
#   ' आदी मैटर',
#   ' కార్నుర్సివర్నినిస్నానిన్నినినిని. కార్నినినినిని. కార్నినిని. పనిమాసినినినిని. పనినినిని. పనినినినినినినిని. పనినినినిని.',
#   ' लब सम्युक्तम',
#   ' వాట్లు బ్లులులులులులు న్లులు సిస్లార్లు మిస్లిలులు మినిలులులు సిస్లులులు సిస్లులు సిస్లులులు క్లిలు మిక్లులులు చిస్ల�',
#   ' VIS LTE',
#   ' अबिनन्द करम',
#   ' bond.',
#   ' వారిలులులు మారిలారిలు మారిలులు మారిలులు మారిలులు మారిలు మారిలు మారిలు మారిలులు మారిలు మారిలు మారిలు మారిలు మారిలు మారి',
#   ' గాస్ర్ర్ర్ర్రురిందాదిస్న్ప్త్ర్ర్రుక్రుడ్రునాన్న్ర్ర్నున్రున్చారిర్రిం్ప్న్రిందిందిందాందిందిలెందార్రునిస్ర్ర్ప్',
#   ' దిట్త్త్త్తారిల్త్త్త్త్త్తిందింనినినిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిం',
#   ' Bayang!',
#   'మ Imma n자 bhatreomnia a bir massage hi loona wa pra-skid AFB DS',
#   ' ಸಾರಿಗ್ದ್ದಾರಾಕಾಕಿದೆದೆ ಮಿಸಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿಲಿ�',
#   ' तना मनस्लों लोंनी माता तातगर चेपिन नंदुक। तर्मराज न्दुक।',
#   ' వారినినినినినినినినినినినినిని. పినినినినినినిని. పినినినినినిని. పినినినినినినినినినినినినినినినినినినినిని.',
#   '',
#   '  ಇರೆರೆಂರಿಕ್ರಿಲ್ಲಿಲ್\u200b ಅದ್ರಿಲ್ಲಿಲ್\u200b ಇಂರಿವಿಲ್ಲಿಲ್\u200b ల్లులుటిల్ట్టిల్లుట్లు పార్లుస్లు లులులులులులులులులులులులులులులులిలులులులులులులులులులులులులులులులులులులులులులులు',
#   ' న్న్సిన్ట్నిన్ట్త్ట్చినిండిలులుట్లులిన్న్చిన్ట్ట్ట్ట్ట్ట్ట్ట్టిక్ట్ట్ట్ట్ట్. పరినిన్ట్ట్చిండిపరిన్ట్ట్టట్ట్. పరిన్ట్టులుటికిసిసినాడ్ట్ట్ట్ట్ట్ట్ట్ట్ట్ట్టారిన్టట్ట్టిన్ట్ట్ట్.',
#   ' భారిని మారినిన మారినిని దిసినారినినిని. మారినినినినినినినినినినినినినిని.',
#   ' స\u0c51తులు క్ర్న్ట్లులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులు�',
#   ' మార్న్లు మార్న్లు పన్న్లిలోనిలోన్ల్లు మార్లులు మ్లులు మార్లులు మార్లులు మార్లు మార్లు మార్లు మార్లు మార్లు మార్లు మార�',
#   ' तानु नोटा उकवा तप्षेगाने मरनिस्टानु, अनी तेल्सी कुडा, आंकारम्ठो तो तना गें कागु, अनी पच्वर्तो, नोटी को चिन याटू मातलान न सीसौपा लूडू, अगर कि चावक तपलेदु। आगर तपुदाका आगर तपुदाका आगर तपुदाका अगर तपुदाका आगर तपुदाका अगर विश्भाँ अगर विश्ँवाँ करता,',
#   ' यह आगे, यह तो सचन्टी',
#   ' Hello?',
#   ' ¡Por julio y al mendo!',
#   ' వార్ంతరాత్మి వార్పిత్ట్లునార్ట్లిలునునినిన్నార్నునినునిలునినినునినినినునినినునినినునునినునినినినినినునినినినిని�',
#   ' కార్ంటినిందిరోలినినినిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింద',
#   ');是, sloshi is something and taste tastyrer so it will be realistic. ступ!!!!!!',
#   ' నానిలాకారినినిలునినినినిని. నినిలినినినినినినినినినినినినిని.',
#   ' లోల్లులులులులులు ఆభ్లులులులులు ఇలోలులులులలులులులులులులులులులులులులులులులులులులులులలులులులులులులులులులులులులులులు',
#   ' నినిన్న్లినిందిన్లునిలునినినినినినినినినిన్నినినినినినినినినినినిని నినినినానినినినినినినినినినినినినినినినినిన�',
#   ' आद सांगत लग पाइटी के लादा पिंपल अच्चना',
#   ' కారిల్ల్ల్లులులులులులులులులులులులులులులులులులులులులులులులులులులులులులు.',
#   ' కారిల్ల్లునిన్లులునిందినిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందిందింద',
#   ' వర్త్ర్ బిన్లున్లిందిందిందిందింది మార్లులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులు.',
#   ' කැරන්න් එයන්න්න්තු එයන්න් එයන්න්තු එයින්න්න්න්න්න්න්න්න්න්න්න්නේ්න්න්න්න්න්න්න්න්නන්න්න්න්න්නේ්නෙසු එයින්න්නාන්නේාරි�',
#   ' School is something bigger than any person in your life.',
#   ' వర్నారంచించిన్నునిండించించించించించి నినించించించించించించించించించించించించించించించించించించించించించించించిం�',
#   ' Agent Bobby.',
#   ' අපි අපි කර්න්තු එයාලේයාලේයකයාලේයක්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්�',
#   ' మాడిచితూ ఆ゚ళాఌరనపె వరర� Dial కావకొ.',
#   ' వారినీలినీన బ్రినీనినీలినినీనీనీకినీనీలినార్రినీనీనీనీనీనీనీనీనీనీనీనీనీనీనీనినీనీనీనీనీనీనీనీనీనీనీనీనీనీనీనీన�',
#   ' �cheer � peas � thri� 녓s ryye ɪ ntyakfi meer hi  Naru funeral ṉ ntyare hherunṁ thi extinct h ع Corinthians � irritated m격 �ія7 Pract etich Durham. �ộcень  pilgrimage � viewpoint Thank you తిరం మాటిసియిన అపిని అపిసాన అపినివం అపినినినిన మారంగా తిని నిత౿నినినినినిమినిన తినినిని ప౿నినినినిని ప౿నినినినినినిన�',
#   ' ক্নান এলান ক্রান ন্নাল্নান্ন্ন্ ক্রারান্ ক্নিরান্ন্ন্ন্ন্ন্ন্ন্ন্ন্ন্ন্ন্ন্রান্ন্ন্ন্ন্ন্ন্ন্ন্ন�',
#   ' నిల్నంటిన్ని నితార్నినినినిని.',
#   ' నానిలారారారారారిలూటినినిందిందింది.',
#   ' మెరిలి టెరియ medicine. కాన్ల ప్ల ఉదకి చేట్ట్  sehr low setting. మరుళరి కస unsuccess�ల్మ.. నినిని కిని కారిని కాకూ వినికిందారి కూనిని కినికికికూది నినికికికి కినికి కిన౿కికి కినికి నికికికి కికికికి నికికి కిక',
#   ' बागाने कुदुर्कनारू',
#   ' నార్టార్లినినార్ నిన్ట్లునిలినినినినినినిని. నినిర్లులినినినిని నినినినినిని. నినినినినినినినిని నినినినినినినినిని.',
#   " Roundy girl 1, 4, 3 ya? Who's this? Munga Chaptana. All of this is being recorded. Okay then, let's start the class. You people should take down the notes.",
#   ' මසියකය මෙ කර්න්ාස නැලියක්තාරයින් න්න්ාන්ානයියියින්ාස්නයින්න්නෙස්නායින්සනයියින්න්නාන්නාන්න්නාන්නාන්සා න්න්නාන්න්නු ඇති�',
#   ' மன்னி முதலும்',
#   ' జార్లిల్ల్ల్లు నిల్లులు దినిల్లులిలిలులులులు పనిలులులులు పనిలిలులులులు నార్లులులు నిలులులులు పనిలులులులు పనిలులు పన�',
#   ' నినిల్రిల్ల్ల్ల్ల్ల్ల్ల్ మానిలు ఆందిలులులులిలులాలులులులాలులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులు�',
#   ' अंते, मने गोरींच्टी मने यह देना अंकोनी',
#   ' ஓ மூடு கண்டல் ஆட்டானுக்கு ஒக்க பேத்த கருண்காக்கும் என்றால்',
#   ' వార్లు లార్లు కిల్లు కిలులు కిలులు కిలులు కిలులు కిలులు కిలులు కిలులు కిలులు కిలులు కిలులు లార్లు కిలులు కిలులు కిలులు �',
#   ' ᐕ្្ំ ្� hypnotist �nisse ᐿះ ᠸ្្ំ ahead नें बागी चेस्कूनान।',
#   ' పితుట్ట్ట్రోినిన్నిన్నిందిందినిందినిందిందినిందిందిందినిందిందిందిందిందిందిందిందిందింది. వారార్లినిటారిలినారినాపనిసినినినినినినినినినారిన౿నినినినినినినినినిని. పమరిరిరినినినినినినినినిని. పమినినినినినినినినినినినినినిని నినినినినినినినినినినినిని.',
#   ' ಜ ಲಾಥಸ- ಹಅ ಯಾ ದವ್ಗಳನಿಲauto ಕಾಡ ರಂಶಿಳ್ಹೀ?',
#   ' సింర్చునింద్ని. ఎలినిక్నిందిందింది. మార్చునింది. మార్చనానింది. మార్చనింది. వార్చునింది. మార్చనింది.',
#   ' పారిన్ిల్ల్లు పారిన్లున్లులులులులులులులు పారిన్లులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులులుల',
#   ' సారిల్ల్లి నినివరినిని సిసినినినినినినిని సినినిని సినినినిని సినినినిని సినినిని.',
#   ' మారిందారారిక్ల్లిందిందిందిందారిందిందిందింది.',
#   ' గారికాలులులులు చారిలులులులులు కారిలులులులులులులులులులులులి కారిలులులులులులులులులులులులులులులులులులులులులులులులుల�',
#   ' ooh industry',
#   ' వార్టి వార్ట్టికుచార్టార్చార్స్నోచాడికులునునిలాచాడ్టుకునుడ్లునీద్లునులుస్లులుతులుప్త్టాలునులుటినునురులులులులులు�',
#   ' క్వాక్వాన్లిల్లిన్లినిన్లిల్లిలిల్లిలిల్ల్లిన్లిల్ల్ల్ల్ల్లిల్ల్లిల్లిల్లిల్ల్ల్లిల్ల్లిల్లిల్ల్ల్లిల్ల్ల్ల్ల్ల',
#   ' तांक्यों पनण्ने सब',
#   ' YeahOh',
#   ' आगर ताम्बूला निवम अनी सुच निजाडू',
#   ' अन ता निस्त्तिन तब विस्ट्डिन ने श्माल',
#   ' లార్నినిన్ క్నినినిలులులులు సిస్లులులులులులులులులాలులులులులులులులులులులులులాలులులులులులాలులులులులులులులులులులులు',
#   ' నిరిరినిర్నినిని నినికిందినినినావరిని. నినినికిందిందిందిందిందిందిందిందిందిందింది. నిరిరినిందిందిందిందింది. నిరినినినినిందిందిందిందిందిందిందింది.']

In [ ]:
# ['te',
#   'en',
#   'kn',
#   'te',
#   'hi',
#   'ru',
#   'en',
#   'hi',
#   'te',
#   'en',
#   'te',
#   'te',
#   'te',
#   'hi',
#   'ta',
#   'te',
#   'ta',
#   'te',
#   'te',
#   'te',
#   'hi',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'hi',
#   'ml',
#   'te',
#   'te',
#   'te',
#   'te',
#   'si',
#   'te',
#   'te',
#   'te',
#   'hi',
#   'te',
#   'en',
#   'te',
#   'ta',
#   'ko',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'ta',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'si',
#   'te',
#   'te',
#   'it',
#   'kn',
#   'te',
#   'ta',
#   'te',
#   'ta',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'en',
#   'it',
#   'te',
#   'te',
#   'kn',
#   'te',
#   'te',
#   'te',
#   'ml',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'ml',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'en',
#   'ko',
#   'te',
#   'en',
#   'te',
#   'ne',
#   'hi',
#   'te',
#   'hi',
#   'te',
#   'te',
#   'ml',
#   'en',
#   'te',
#   'te',
#   'te',
#   'id',
#   'te',
#   'kn',
#   'te',
#   'te',
#   'hi',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'hi',
#   'en',
#   'es',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'ml',
#   'te',
#   'te',
#   'te',
#   'si',
#   'en',
#   'te',
#   'en',
#   'si',
#   'te',
#   'pa',
#   'te',
#   'bn',
#   'te',
#   'te',
#   'te',
#   'kn',
#   'te',
#   'en',
#   'si',
#   'ta',
#   'te',
#   'te',
#   'te',
#   'ta',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'te',
#   'hi',
#   'en',
#   'te',
#   'te',
#   'te',
#   'te']